# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
T. Mikal-Evans  ->  T. Mikal-Evans  |  ['T. Mikal-Evans']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 30 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2401.01452


extracting tarball to tmp_2401.01452...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

T. Mikal-Evans  ->  T. Mikal-Evans  |  ['T. Mikal-Evans']


Found 158 bibliographic references in tmp_2401.01465/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2401.01517


extracting tarball to tmp_2401.01517...

 done.
Retrieving document from  https://arxiv.org/e-print/2401.01829


extracting tarball to tmp_2401.01829... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.01452-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.01452) | **Euclid preparation: TBD. The pre-launch Science Ground Segment  simulation framework**  |
|| E. Collaboration, et al. -- incl., <mark>M. Schirmer</mark> |
|*Appeared on*| *2024-01-04*|
|*Comments*| *38 pages, 25 figures, A&A submitted*|
|**Abstract**| The European Space Agency's Euclid mission is one of the upcoming generation of large-scale cosmology surveys, which will map the large-scale structure in the Universe with unprecedented precision. The development and validation of the SGS pipeline requires state-of-the-art simulations with a high level of complexity and accuracy that include subtle instrumental features not accounted for previously as well as faster algorithms for the large-scale production of the expected Euclid data products. In this paper, we present the Euclid SGS simulation framework as applied in a large-scale end-to-end simulation exercise named Science Challenge 8. Our simulation pipeline enables the swift production of detailed image simulations for the construction and validation of the Euclid mission during its qualification phase and will serve as a reference throughout operations. Our end-to-end simulation framework starts with the production of a large cosmological N-body & mock galaxy catalogue simulation. We perform a selection of galaxies down to I_E=26 and 28 mag, respectively, for a Euclid Wide Survey spanning 165 deg^2 and a 1 deg^2 Euclid Deep Survey. We build realistic stellar density catalogues containing Milky Way-like stars down to H<26. Using the latest instrumental models for both the Euclid instruments and spacecraft as well as Euclid-like observing sequences, we emulate with high fidelity Euclid satellite imaging throughout the mission's lifetime. We present the SC8 data set consisting of overlapping visible and near-infrared Euclid Wide Survey and Euclid Deep Survey imaging and low-resolution spectroscopy along with ground-based. This extensive data set enables end-to-end testing of the entire ground segment data reduction and science analysis pipeline as well as the Euclid mission infrastructure, paving the way to future scientific and technical developments and enhancements. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.01465-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.01465) | **Is the atmosphere of the ultra-hot Jupiter WASP-121b variable?**  |
|| Q. Changeat, et al. -- incl., <mark>T. Mikal-Evans</mark> |
|*Appeared on*| *2024-01-04*|
|*Comments*| *Accepted for publication in ApJS. 43 pages, 31 figures, 2 animations (available online at the journal)*|
|**Abstract**| We present a comprehensive analysis of the Hubble Space Telescope observations of the atmosphere of WASP-121 b, a ultra-hot Jupiter. After reducing the transit, eclipse, and phase-curve observations with a uniform methodology and addressing the biases from instrument systematics, sophisticated atmospheric retrievals are used to extract robust constraints on the thermal structure, chemistry, and cloud properties of the atmosphere. Our analysis shows that the observations are consistent with a strong thermal inversion beginning at ~0.1 bar on the dayside, solar to subsolar metallicity Z (i.e., -0.77 < log(Z) < 0.05), and super-solar C/O ratio (i.e., 0.59 < C/O < 0.87). More importantly, utilizing the high signal-to-noise ratio and repeated observations of the planet, we identify the following unambiguous time-varying signals in the data: i) a shift of the putative hotspot offset between the two phase-curves and ii) varying spectral signatures in the transits and eclipses. By simulating the global dynamics of WASP-121 b atmosphere at high-resolution, we show that the identified signals are consistent with quasi-periodic weather patterns, hence atmospheric variability, with signatures at the level probed by the observations (~5% to ~10%) that change on a timescale of ~5 planet days; in the simulations, the weather patterns arise from the formation and movement of storms and fronts, causing hot (as well as cold) patches of atmosphere to deform, separate, and mix in time. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.01517-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.01517) | **The rotation curve and mass distribution of M31**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2024-01-04*|
|*Comments*| **|
|**Abstract**| To gain a better understanding of the Andromeda galaxy M31 and its role in the Local Group, measuring its mass precisely is essential. In this work, we have constructed the rotation curve of M31 out to $\sim$125 kpc using 13,679 M31 objects obtained from various sources, including the LAMOST data release 9 (LAMOST DR9), the DESI survey, and relevant literature. We divide all objects in our sample into bulge, disk and halo components. For the sources in the M31 disk, we have measured their circular velocities by a kinematic model with asymmetric drift corrections. For the bulge and halo objects, we calculate their velocity dispersions and use the spherical and projected Jeans equation to obtain the circular velocities. Our findings indicate a nearly isotropic nature for the M31 bulge, while the halo exhibits tangential anisotropy. The results show that the rotation curve remains constant at $\sim$220 km s$^{-1}$ up to radius $\sim$25 kpc and gradually decreases to $\sim$170 km s$^{-1}$ further out. Based on the newly determined rotation curve, we have constructed a mass distribution model for M31. Our measurement of the M31 virial mass is $M_{\rm vir} = 1.14^{+0.51}_{-0.35} \times 10^{12} M_\odot$ within $r_{\rm vir} = 220 \pm 25$ kpc. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.01829-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.01829) | **Constraints on Axion-like Particles from the Observation of Galactic  Sources by LHAASO**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2024-01-04*|
|*Comments*| **|
|**Abstract**| High-energy photons may oscillate with axion-like particles (ALPs) when they propagate through the Milky Way's magnetic field, resulting in an alteration in the observed photon energy spectrum. The ultra-high energy gamma-ray spectra, measured by the Large High Altitude Air Shower Observatory (LHAASO) up to $\mathcal{O}(1)~\mathrm{PeV}$, provide a promising opportunity to investigate the ALP-photon oscillation effect. In this study, we utilize the gamma-ray spectra of four Galactic sources measured by LHAASO, including the Crab Nebula, LHAASO J2226+6057, LHAASO J1908+0621, and LHAASO J1825-1326, to explore this effect. We employ the $\rm CL_s$ method to set constraints on the ALP parameters. Combing the observations of the four sources, our analysis reveals that the ALP-photon coupling $g_{a\gamma}$ is constrained to be smaller than $1.4\times10^{-10}$ ${\rm GeV}^{-1}$ for the ALP mass of $\sim 4\times10^{-7} ~\mathrm{eV}$ at the 95\% C.L. By combing the observations of the Crab Nebula from LHAASO and other experiments, we find that the ALP-photon coupling could be set to be about $7.2\times10^{-11}$ ${\rm GeV}^{-1}$ for the ALP mass $\sim 4 \times10^{-7}~\mathrm{eV}$ , which is in close proximity to the CAST constraint. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2401.01452.md
    + _build/html/tmp_2401.01452/./star_catalogs_SC8_1.png
    + _build/html/tmp_2401.01452/./euclid_transmissions_v2_schirmer.png
    + _build/html/tmp_2401.01452/./Cosmos_seds.png
exported in  _build/html/2401.01465.md
    + _build/html/tmp_2401.01465/./spectra_tp_retrieval.png
    + _build/html/tmp_2401.01465/./tp_retrievals_1d.png
    + _build/html/tmp_2401.01465/./tp_1d_test1.png
    + _build/html/tmp_2401.01465/./transit_spectra_origin.png
    + _build/html/tmp_2401.01465/./eclipse_spectra_origin.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\dotdeg}{\rlap{.}^\circ}$
$\newcommand{\orcid}[1]$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# Euclid preparation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.01452-b31b1b.svg)](https://arxiv.org/abs/2401.01452)<mark>Appeared on: 2024-01-04</mark> -  _38 pages, 25 figures, A&A submitted_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>M. Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** The European Space Agency's _Euclid mission_ is one of the upcoming generation of large-scale cosmology surveys, which will map the large-scale structure in the Universe with unprecedented precision. The mission will collect vast amount of data that will be processed and analysed by $\Euclid$ 's Science Ground Segment (SGS). The development and validation of the SGS pipeline requires state-of-the-art simulations with a high level of complexity and accuracy that include subtle instrumental features not accounted for previously as well as faster algorithms for the large-scale production of the expected $\Euclid$ data products. In this paper, we present the $\Euclid$ SGS simulation framework as applied in a large-scale end-to-end simulation exercise named Science Challenge 8. Our simulation pipeline enables the swift production of detailed image simulations for the construction and validation of the $\Euclid$ mission during its qualification phase and will serve as a reference throughout operations. Our end-to-end simulation framework starts with the production of a large cosmological N-body simulation which we use to construct a realistic galaxy mock catalogue. We perform a selection of galaxies down to $\IE$ =26 and 28 mag, respectively, for a Euclid Wide Survey spanning $165 {\rm deg}^2$ and a $1 {\rm deg}^2$ Euclid Deep Survey. We build realistic stellar density catalogues containing Milky Way-like stars down to $H<26$ produced from a combination of a stellar population synthesis model of the Galaxy and real bright stars. Using the latest instrumental models for both the $\Euclid$ instruments and spacecraft as well as $\Euclid$ -like observing sequences, we emulate with high fidelity $\Euclid$ satellite imaging throughout the mission's lifetime. We present the SC8 data set consisting of overlapping visible and near-infrared Euclid Wide Survey and Euclid Deep Survey imaging and low-resolution spectroscopy along with ground-based data in five optical bands.  This extensive data set enables end-to-end testing of the entire ground segment data reduction and science analysis pipeline as well as the $\Euclid$ mission infrastructure, paving the way to future scientific and technical developments and enhancements.

</div>

<div id="div_fig1">

<img src="tmp_2401.01452/./star_catalogs_SC8_1.png" alt="Fig13" width="100%"/>

**Figure 13. -** Density of objects (number of stars per ${\rm deg}^2$) in each standard star catalogues file covering SC8 Main Production area, superimposed with the \Euclid observations and simulated EXT survey area. (*fig:star_catalogues_sc8*)

</div>
<div id="div_fig2">

<img src="tmp_2401.01452/./euclid_transmissions_v2_schirmer.png" alt="Fig1" width="100%"/>

**Figure 1. -** The total transmission of the VIS (\IE) and NISP photometric  (\YE, \JE, \HE), and the NISP Spectroscopic ($BG_E$, $RG_E$) bands. (*fig:euclid_transmissions*)

</div>
<div id="div_fig3">

<img src="tmp_2401.01452/./Cosmos_seds.png" alt="Fig4" width="100%"/>

**Figure 4. -** The reference template spectral energy distributions, used in combination with extra extinction and emission line prescriptions in order to reconstruct complete spectra. (*fig:cosmos_seds_dust*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.01452"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ipwtodo}{\todo[author=IPW,inline,size=\footnotesize,color=blue!30!white]}$
$\newcommand{\gttodo}{\todo[author=GT,inline,size=\footnotesize,color=green!30!white]}$
$\newcommand{\betodo}{\todo[author=BE,inline,size=\footnotesize,color=yellow!30!white]}$
$\newcommand{\qctodo}{\todo[author=QC,inline,size=\footnotesize,color=red!30!white]}$
$\newcommand{\hi}[1]{\textcolor{cyan}{#1}}$
$\newcommand{\rem}[1]{\textcolor{red}{[#1]}}$
$\newcommand{\remQ}[1]{\textcolor{purple}{[#1]}}$
$\newcommand{\thefootnote}{\fnsymbol{footnote}}$
$\newcommand{\thefootnote}{\arabic{footnote}}$
$\newcommand$
$\newcommand$
$\newcommand{\thefigure}{A\arabic{figure}}$
$\newcommand{\theHfigure}{A\arabic{figure}}$
$\newcommand{\floatpagefraction}{.99}$
$\newcommand{\thefigure}{B\arabic{figure}}$
$\newcommand{\theHfigure}{B\arabic{figure}}$
$\newcommand{\thefigure}{D\arabic{figure}}$
$\newcommand{\theHfigure}{D\arabic{figure}}$
$\newcommand{\thefigure}{E\arabic{figure}}$
$\newcommand{\theHfigure}{E\arabic{figure}}$</div>



<div id="title">

# Is the atmosphere of the ultra-hot Jupiter WASP-121 b variable?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.01465-b31b1b.svg)](https://arxiv.org/abs/2401.01465)<mark>Appeared on: 2024-01-04</mark> -  _Accepted for publication in ApJS. 43 pages, 31 figures, 2 animations (available online at the journal)_

</div>
<div id="authors">

Q. C. $^\dagger$, et al. -- incl., <mark>T. Mikal-Evans</mark>

</div>
<div id="abstract">

**Abstract:** We present a comprehensive analysis of the Hubble Space Telescopeobservations of the atmosphere of WASP-121 b, a ultra-hot Jupiter.After reducing the transit, eclipse, and phase-curve observationswith a uniform methodology and addressing the biases frominstrument systematics, sophisticated atmospheric retrievals areused to extract robust constraints on the thermal structure,chemistry, and cloud properties of the atmosphere.Our analysis shows that the observations are consistent with astrong thermal inversion beginning at $\sim10^4$ Pa on thedayside, solar to subsolar metallicity $Z$ (i.e., $-0.77 < \log(Z) < 0.05$ ), and super-solar C/O ratio(i.e., $0.59 < \textrm{C/O} < 0.87$ ).More importantly, utilizing the high signal-to-noise ratio andrepeated observations of the planet, we identify the followingunambiguous time-varying signals in the data: $_ i_$ ) a shift ofthe putative $_ hotspot_$ offset between the two phase-curves and $_ ii_$ ) varying spectral signatures in the transits and eclipses.By simulating the global dynamics of WASP-121 batmosphere at high-resolution, we show that the identified signalsare consistent with quasi-periodic weather patterns, henceatmospheric variability, with signatures at the level probed bythe observations ( $\sim$ 5 \% to $\sim$ 10 \% ) that change on atimescale of $\sim$ 5 planet days; in the simulations, theweather patterns arise from the formation and movement of stormsand fronts, causing hot (as well as cold) patches of atmosphereto deform, separate, and mix in time.

</div>

<div id="div_fig1">

<img src="tmp_2401.01465/./spectra_tp_retrieval.png" alt="Fig16" width="100%"/>

**Figure 16. -** Recovered temperature--pressure ($T$--$p$) profiles
    (left) and best-fit spectra (right) for the phases from 0.05
    (blue) to 0.5 (red), obtained from the phase-curve
    atmospheric retrieval.
    In the $T$--$p$ plot, the shaded regions correspond to one
    and three sigma confidence regions (dark to light,
    respectively).
    The  radiative contribution function is also shown in
    dashed line, colored for each region: hotspot (red),
    dayside (orange), and nightside (blue).
    These retrievals show good agreement with the observed data
    and demonstrate a strong dayside thermal inversion, with
    the presence of a hotter region (e.g. hotspot).
    The best-fit $T-p$ profiles (solid lines, left) are used
    to thermally force the atmospheric dynamics simulations.
    This figure is accompanied by a 15 s video, available online at the journal, showing the evolution of WASP-121 b emission (from blue to red) and the corresponding thermal structure as a function of phase. As the planet moves from transit to eclipse, absorption features in the data are replaced by emission features. These spectral variations enable the characterization of the thermal structure and chemistry across WASP-121 b's atmosphere. (*fig:spectra_tp_retrieval*)

</div>
<div id="div_fig2">

<img src="tmp_2401.01465/./tp_retrievals_1d.png" alt="Fig19.1" width="50%"/><img src="tmp_2401.01465/./tp_1d_test1.png" alt="Fig19.2" width="50%"/>

**Figure 19. -** One-dimensional (1D) thermal structure recovered by
    our retrieval analysis of the five eclipse observations with
    one sigma confidence region (left), and $T$--$p$ profiles
    from multiple times ($t \in[40, 185]$ days) at the substellar
    point from a three-dimensional (3D) atmospheric dynamics
    simulation (right).
    The magnitude of variability in $p \in[10^5, 10^3]$ Pa is
    $\sim$300 K, which is consistent with the variation predicted
    by the 3D simulation.
    Dashed gray lines show the vertical extent of the atmosphere
    modeled by the simulations in this study.
    Note, while these profiles are not like-for-like comparable
    because the retrieved thermal structure is global and
    substellar temperature predictions are local.  (*fig:tp1d*)

</div>
<div id="div_fig3">

<img src="tmp_2401.01465/./transit_spectra_origin.png" alt="Fig10.1" width="50%"/><img src="tmp_2401.01465/./eclipse_spectra_origin.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** 
    Transit (left) and eclipse (right) spectra of WASP-121 b analyzed
    in this work. Different observations are offset in the $y$-axis.
    Best-fit models from the 1D retrievals are shown in solid lines.
    Dashed lines show featureless models for visual comparison. (*fig:spectra_ec_tr*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.01465"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

350  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
